In [21]:
import pandas as pd
import sklearn as sl
import quandl

df = quandl.get('WIKI/GOOGl')
print(df.head())

LimitExceededError: (Status 429) (Quandl Error QELx01) You have exceeded the anonymous user limit of 50 calls per day. To make more calls today, please register for a free Quandl account and then include your API key with your requests.

In [0]:
df = df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume',]]
df.head()

In [0]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100

In [0]:
df['PCT_change'] = (df['Adj. Close']- df['Adj. Open']) / df['Adj. Open'] * 100

In [0]:
df = df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]
df

In [0]:
forecast_col = 'Adj. Close'
df.fillna(-99999, inplace=True)


In [0]:
import math
forecast_out = int(math.ceil(0.01*len(df)))
print(forecast_out)


In [0]:
df['label'] = df[forecast_col].shift(-forecast_out)

print(df.head())

In [0]:
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

X = np.array(df.drop(['label'],1))
X =  preprocessing.scale(X)
X = X[:-forecast_out]
X_lately = X[-forecast_out:]

df.dropna(inplace=True)

y = np.array(df['label'])
# y = np.array(df['label'])



In [0]:
print(len(X),len(y))

In [0]:
X_train, X_test, y_train,y_test = model_selection.train_test_split(X,y, test_size=0.2)
clf = LinearRegression()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test,y_test)



In [0]:
forecast_set = clf.predict(X_lately)
print(forecast_set,accuracy, forecast_out)

In [0]:
print(accuracy)
print(forecast_out)

In [0]:
clf = svm.SVR()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test,y_test)


In [0]:
print(accuracy, forecast_out)

In [0]:
import matplotlib.pyplot as plt
import datetime
from matplotlib import style

style.use('ggplot')

In [0]:
df['Forecast'] = np.nan

In [0]:
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day


In [0]:
for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]

In [0]:
import pickle

with open('linearregression.pickle','wb') as f:
    pickle.dump(clf,f)

pickle_in = open('linearregression.pickle','rb')
clf = pickle.load(pickle_in)

In [0]:
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend(loc=4)
plt.show()